## Load data

In [54]:
from PIL import Image
import numpy as np
from numpy import asarray
#def transformImage(filename):
    
def getInputs():
    imgs = []
    for i in range(1,101):
        filename = 'test_image_png_'+str(i)+'.png'
        img = Image.open(filename).resize((8,8))
        imgs.append(img)
    pxMatrices = []
    for img in imgs:
        pxMatrices.append(np.array(img.getdata()))
    return pxMatrices
        
inputs = getInputs()
outputs = []
for i in range(0,100):
    outputs.append('0')
outputNames = ['0','1']





### separating training data from test data 20%-80%

In [55]:
import numpy as np 

def splitData(inputs, outputs):
    np.random.seed(5)
    indexes = [i for i in range(len(inputs))]
    trainSample = np.random.choice(indexes, int(0.8 * len(inputs)), replace = False)
    testSample = [i for i in indexes  if not i in trainSample]

    trainInputs = [inputs[i] for i in trainSample]
    trainOutputs = [outputs[i] for i in trainSample]
    testInputs = [inputs[i] for i in testSample]
    testOutputs = [outputs[i] for i in testSample]
    
    return trainInputs, trainOutputs, testInputs, testOutputs




**normalize data**

In [56]:
from sklearn.preprocessing import StandardScaler

def normalisation(trainData, testData):
    scaler = StandardScaler()
    if not isinstance(trainData[0], list):
        #encode each sample into a list
        trainData = [[d] for d in trainData]
        testData = [[d] for d in testData]
        
        scaler.fit(trainData)  #  fit only on training data
        normalisedTrainData = scaler.transform(trainData) # apply same transformation to train data
        normalisedTestData = scaler.transform(testData)  # apply same transformation to test data
        
        #decode from list to raw values
        normalisedTrainData = [el[0] for el in normalisedTrainData]
        normalisedTestData = [el[0] for el in normalisedTestData]
    else:
        scaler.fit(trainData)  #  fit only on training data
        normalisedTrainData = scaler.transform(trainData) # apply same transformation to train data
        normalisedTestData = scaler.transform(testData)  # apply same transformation to test data
    return normalisedTrainData, normalisedTestData



    **ANN Classification (logic)**

In [57]:
from sklearn import neural_network

classifier = neural_network.MLPClassifier()

def training(classifier, trainInputs, trainOutputs):
    # step4: training the classifier
    # identify (by training) the classification model
    classifier.fit(trainInputs, trainOutputs)

def classification(classifier, testInputs):
    # step5: testing (predict the labels for new inputs)
    # makes predictions for test data 
    computedTestOutputs = classifier.predict(testInputs)

    return computedTestOutputs
            

### leran model

In [58]:

        

trainInputs, trainOutputs, testInputs, testOutputs = splitData(inputs, outputs)
def flatten(mat):
    x = []
    for line in mat:
        if isinstance(line,list):
            for el in line:
                x.append(el)
    return x 

trainInputsFlatten = [flatten(el) for el in trainInputs]
testInputsFlatten = [flatten(el) for el in testInputs]
trainInputsNormalised, testInputsNormalised = normalisation(trainInputsFlatten, testInputsFlatten)
classifier = neural_network.MLPClassifier(hidden_layer_sizes=(5,), activation='relu', max_iter=100, solver='sgd', verbose=10, random_state=1, learning_rate_init=.1)

training(classifier, trainInputsNormalised, trainOutputs)
predictedLabels = classification(classifier, testInputsNormalised)


ValueError: Found array with 0 feature(s) (shape=(80, 0)) while a minimum of 1 is required by StandardScaler.

## predict classes for test data

In [ ]:
def evalMultiClass(realLabels, computedLabels, labelNames):
    from sklearn.metrics import confusion_matrix

    confMatrix = confusion_matrix(realLabels, computedLabels)
    acc = sum([confMatrix[i][i] for i in range(len(labelNames))]) / len(realLabels)
    precision = {}
    recall = {}
    for i in range(len(labelNames)):
        precision[labelNames[i]] = confMatrix[i][i] / sum([confMatrix[j][i] for j in range(len(labelNames))])
        recall[labelNames[i]] = confMatrix[i][i] / sum([confMatrix[i][j] for j in range(len(labelNames))])
    return acc, precision, recall, confMatrix
acc, precision, recall, cm = evalMultiClass(np.array(testOutputs), predictedLabels, outputNames)

### calculate error



In [ ]:
print('acc: ', acc)
print('precision: ', precision)
print('recall: ', recall)
